In [56]:
# Importing necessary libraries
import pandas as pd

# Re-running the steps with the provided code

# Step 1: Read the Excel file and extract the required sheets
roles_df = pd.read_excel("FantaSPL_V2.2.xlsx", sheet_name=3)
data_first_sheet = pd.read_excel("FantaSPL_V2.2.xlsx", sheet_name=0)
punti_df = pd.read_excel("FantaSPL_V2.2.xlsx", sheet_name="Punti")

# Provided dictionaries for translations
translation_dict = {
    "Difensore": "Defender",
    "Centrocampista": "Midfielder",
    "Attaccante": "Forward",
    "Portiere": "Goalkeeper"
}
column_mapping_first_sheet = {
    "Data": "Date",
    "Tipo Partita": "Match Type",
    "Bianchi": "Team A Goals",
    "Scuri": "Team B Goals",
    "Squadra vincente": "Winning Team",
    "Numero Goal": "Number of Goals",
    "Goal Check": "Goal Check",
    "Numero Goal Punti": "Goal Points",
    "Autogoal": "Own Goals",
    "Costo per Partita": "Cost per Match",
    "Effettivo": "Actual Cost"
}
translations = {
    "Data": "Date",
    "Player": "Player",
    "Ruolo": "Role",
    "Colore Squadra": "Team Color",
    "Vincitori": "Winners",
    "Vinta": "Won",
    "#Goal": "Goals",
    "Goal Subiti": "Goals Conceded",
    "Goal Fatti": "Goals Scored",
    "AutoGoal": "Own Goals",
    "SPL Bonus": "SPL Bonus",
    "MVP": "MVP",
    "Amici Portati": "Friends Brought",
    "Penalità": "Penalties",
    "Presenza": "Attendance",
    "Penalità.1": "Penalties.1",
    "Punteggio Difesa": "Defense Score",
    "Punteggio Centrocampisti": "Midfielders Score",
    "Squadra Vincente": "Winning Team",
    "#Goal.1": "Goals.1",
    "AutoGoal.1": "Own Goals.1",
    "SPL Bonus.1": "SPL Bonus.1",
    "MVP.1": "MVP.1",
    "Porta Amico": "Friend Referral",
    "TOTAL": "TOTAL"
}

# Step 2: Rename columns for consistency
# For roles_df
roles_df["Position"] = roles_df["Ruolo"].map(translation_dict)

# For data_first_sheet
data_first_sheet.rename(columns=column_mapping_first_sheet, inplace=True)
user_input_table = data_first_sheet[["Date", "Match Type", "Team A Goals", "Team B Goals"]].copy()
user_input_table.loc[:, "Winning Team"] = user_input_table.apply(
    lambda row: "A" if row["Team A Goals"] > row["Team B Goals"] else ("B" if row["Team A Goals"] < row["Team B Goals"] else "X"), axis=1)
user_input_table["Total Goals"] = user_input_table["Team A Goals"] + user_input_table["Team B Goals"]

# For punti_df
punti_df.rename(columns=translations, inplace=True)
user_input_df = punti_df[['Date', 'Player', 'Team Color', 'Goals', 'Own Goals', 'SPL Bonus', 'MVP', 'Friends Brought', 'Penalties']].copy()
user_input_df.loc[:, 'Team Color'] = user_input_df['Team Color'].replace({'Scuri': 'B', 'Bianchi': 'A'})
for column in ['SPL Bonus', 'MVP', 'Penalties']:
    user_input_df.loc[:, column] = user_input_df[column].replace({'Sì': 1, 'No': 0})

# Step 3: Merge the data as needed
# Refactor the game_outcome function
def game_outcome(row):
    if row['Winning Team'] == 'X':
        return 'Draw'
    elif row['Team Color'] == row['Winning Team']:
        return 'Won'
    else:
        return 'Lost'

# More efficient lambda functions and columns creation
merged_table = user_input_df.merge(user_input_table, on='Date', how='inner')
merged_table['Game Outcome'] = merged_table.apply(game_outcome, axis=1)
merged_table['Goals Conceded'] = merged_table.apply(
    lambda row: row['Team B Goals'] if row['Team Color'] == 'A' else row['Team A Goals'],
    axis=1
)
merged_table['Goals Scored'] = merged_table.apply(
    lambda row: row['Team A Goals'] if row['Team Color'] == 'A' else row['Team B Goals'],
    axis=1
)

# Use merge for combining DataFrames on specific columns
merged_table = merged_table.merge(roles_df[['Player', 'Position']], on='Player', how='left')

# Step 4: Ensure that the merged table contains the specified columns
final_columns = ["Date", "Player", "Position", "Team Color", "Game Outcome", "Goals", "Own Goals", "Goals Conceded", "Goals Scored", "SPL Bonus", "MVP", "Friends Brought", "Penalties"]
merged_table = merged_table[final_columns]

# Step 5: Display the head of the final merged table
merged_table.head(10)


,Date,Player,Position,Team Color,Game Outcome,Goals,Own Goals,Goals Conceded,Goals Scored,SPL Bonus,MVP,Friends Brought,Penalties
0,2023-01-12,Andrea Limonta,Defender,B,Lost,0,0,18,13,1,0,0,0
1,2023-01-12,Luca Stoppi,Defender,B,Lost,2,1,18,13,0,0,0,0
2,2023-01-12,Riccardo Ricci,Midfielder,B,Lost,0,0,18,13,0,0,0,0
3,2023-01-12,Marco Taglio,Midfielder,B,Lost,3,0,18,13,0,0,0,0
4,2023-01-12,Mazzu,Midfielder,B,Lost,7,0,18,13,0,0,0,0
5,2023-01-12,Rudy Vitiello,Midfielder,B,Lost,1,0,18,13,0,0,1,0
6,2023-01-12,Carlo Tempesta,Midfielder,B,Lost,0,0,18,13,0,0,0,1
7,2023-01-12,Fabrizio Limonta,Defender,A,Won,0,0,13,18,0,0,0,0
8,2023-01-12,Cameron McAinsh,Midfielder,A,Won,5,0,13,18,0,0,0,0
9,2023-01-12,Giovanni Aiello,Defender,A,Won,1,0,13,18,0,0,0,0
